# Importing, libraries, and formating:

In [3]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
# import scipy.stats as st
# import plotly.express as px

pd.set_option('display.max_columns', None)

In [4]:
# importing and basic formating
flights = pd.read_csv('C:/Users/danfv/OneDrive/Knowledge/Courses/Data Analysis/LHL - Data Analysis Bootcamp/lighthouse-data-notes/W6 Midterm Project/Midterm_Project/Data/Flights - Sample 10000 - Peter Dan Features.csv').sort_values('fl_date').reset_index(drop=True)
flights = flights.drop('Unnamed: 0', axis=1)


# deleting features the model won't use
date_and_carrier = ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num']
origin_and_dest = ['origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name']
drop_streamlining = ['tail_num', 'dup', 'flights']
drop_peter_string = ['dep_weather', 'arr_weather']

initial_drop = date_and_carrier + origin_and_dest + drop_streamlining + drop_peter_string
flights = flights.drop(initial_drop, axis=1)

# renaming target
flights = flights.rename(columns={'total_delay' : 'arr_delay'})

# Separating features and target for regression:
X = flights.drop('arr_delay', axis=1)
y = flights.arr_delay


In [13]:
X

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,dep_weather_rank,arr_weather_rank,origin_airport_freq,dest_airport_freq,season,weekday,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,carrier_enc,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind,orig_airp_enc,dest_airp_enc
0,401,545,845,180,1050,1,1,37,210,1,0,1,0,0.027747,0.066093,-0.037253,10,0.101855,0.00390,0.01688,0.004639,0.016246,23,181
1,487,1445,2215,270,2149,1,1,144,210,1,0,1,0,0.055525,0.032434,-0.037253,10,0.028252,0.01276,0.01688,0.008028,0.016246,261,181
2,3177,2025,2158,93,409,1,1,497,98,1,0,1,0,0.053236,0.043026,-0.001450,0,0.350266,0.04976,0.01044,0.042113,0.008360,217,35
3,5202,2245,20,95,481,1,1,487,35,1,1,1,0,0.032434,0.008157,-0.037253,10,0.350266,0.04904,0.00318,0.035133,0.003390,16,244
4,1393,750,945,115,581,1,1,141,505,1,1,1,0,0.055303,0.062352,-0.037253,10,0.350266,0.01284,0.04904,0.013969,0.035133,108,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4518,1515,1635,80,196,3,1,11,500,1,1,12,1,0.059419,0.056478,0.055916,8,0.350266,0.00148,0.04976,0.001873,0.042113,58,211
9996,3846,1343,1455,72,290,1,2,37,147,1,1,12,1,0.054044,0.055525,-0.001450,0,0.350266,0.00346,0.01660,0.003075,0.020192,218,155
9997,1384,857,1035,98,526,1,1,487,49,1,1,12,1,0.066093,0.059514,-0.019545,3,0.350266,0.04904,0.00656,0.035133,0.007334,16,228
9998,5088,751,854,63,212,1,1,37,173,1,1,12,1,0.055303,0.066093,-0.001450,0,0.213424,0.00346,0.01914,0.003075,0.017167,218,222


In [94]:
flights

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,season,weekday,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,carrier_enc,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind,orig_airp_enc,dest_airp_enc
0,3177,2025,2158,93,409,123.0,1,0,1,0,0.053236,0.043026,-0.001450,0,0.350266,0.04976,0.01044,0.042113,0.008360,218,36
1,487,1445,2215,270,2149,0.0,1,0,1,0,0.055525,0.032434,-0.037253,10,0.028252,0.01276,0.01688,0.008028,0.016246,262,186
2,401,545,845,180,1050,16.0,1,0,1,0,0.027747,0.066093,-0.037253,10,0.101855,0.00390,0.01688,0.004639,0.016246,24,186
3,2475,701,1157,176,1171,0.0,1,1,1,0,0.055303,0.058166,-0.001450,0,0.101855,0.01276,0.03598,0.008028,0.023283,262,79
4,5202,2245,20,95,481,31.0,1,1,1,0,0.032434,0.008157,-0.037253,10,0.350266,0.04904,0.00318,0.035133,0.003390,17,249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1113,1000,1157,117,516,0.0,1,1,12,1,0.059514,0.058166,-0.019545,3,0.225312,0.00346,0.04904,0.003075,0.035133,219,18
9996,3846,1343,1455,72,290,0.0,1,1,12,1,0.054044,0.055525,-0.001450,0,0.350266,0.00346,0.01660,0.003075,0.020192,219,158
9997,243,1445,1804,199,1089,0.0,1,1,12,1,0.055525,0.059659,-0.001450,0,0.101855,0.01660,0.01078,0.020192,0.023044,155,195
9998,801,1324,1610,166,896,0.0,1,1,12,1,0.054044,0.056478,0.001304,4,0.101855,0.00044,0.01688,0.000720,0.016246,299,186


# Defining Function to run linear regressions:

In [10]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor

In [11]:
regressor_list = ['LinearRegression()', 'RandomForestRegressor()']
func_list = ['None', 'np.log']
inverse_func_list = ['None', 'np.exp']

def trans_tar_reg_function(X, y, regressor, func, inverse_func):
    '''
    Generates R2, RMSE and MAE of models using the specified regressor, function and inverse_func for y
    Parameters:
    - X = features df
    - y = target df
    - regressor = method/algo to do the regression
    - func = transformation of y
    - inverse_fun = to get transformed y back to original units
    '''

    #instantiating the target regressor
    ttr = TransformedTargetRegressor(regressor=regressor, func=func, inverse_func=inverse_func)
    # fitting 
    ttr.fit(X, y)
    # Model Evaluation
    y_pred = ttr.predict(X)
    n = y.shape[0]
    p = X.shape[1]
    R2_adj = 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1)
    RMSE = mean_squared_error(y, y_pred)
    MAE = mean_absolute_error(y, y_pred)

    return [R2_adj, RMSE, MAE]

# Feature Selection:

## Handy Feature Lists:

In [6]:
# feature handy lists:
redundant_feat = ['distance']
eng_airp_feat = ['orig_airp_fl_ind', 'dest_airp_fl_ind', 'orig_airp_pss_ind', 'dest_airp_pss_ind']
eng_time_feat = ['season', 'weekday', 'month', 'dep_taxi_ind', 'arr_taxi_ind']
label_enc_feat = ['carrier_enc', 'orig_airp_enc', 'dest_airp_enc']

low_p_val_list = ['season', 'weekday', 'month', 'arr_taxi_ind', 'airtime_ind',
    'orig_airp_fl_ind', 'dest_airp_fl_ind', 'orig_airp_pss_ind', 'dest_airp_pss_ind',
    'origin_airport_freq', 'dest_airport_freq']

In [157]:
# Lists of remainig features after transformations below
print(X_high_var_list)
print(X_fwd_reg_list)
print(X_feat_sel_algo_list)
print(X_feat_sel_algo_randf_list)

['mkt_carrier_fl_num', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance', 'season', 'weekday', 'month', 'peak_season', 'carrier_enc', 'orig_airp_enc', 'dest_airp_enc']
['mkt_carrier_fl_num', 'crs_dep_time', 'crs_arr_time', 'month', 'peak_season', 'dep_taxi_ind', 'arr_taxi_ind', 'carriers_lateness_ind', 'dest_airp_fl_ind', 'dest_airp_pss_ind']
['dep_taxi_ind', 'arr_taxi_ind', 'carriers_lateness_ind', 'orig_airp_fl_ind', 'dest_airp_fl_ind', 'orig_airp_pss_ind']
['mkt_carrier_fl_num', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance', 'dest_airp_pss_ind', 'orig_airp_enc', 'dest_airp_enc']


## Original features:

In [88]:
# original X:
X

# Removing features with p-value > 0.05 when fitting with X (prob. weight = 0 is statistically significant):
# X_pval_filter = X.drop(['Season', 'Weekday', 'dest_airp_fl_ind',
#        'orig_airp_pss_ind', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,season,weekday,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,carrier_enc,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind,orig_airp_enc,dest_airp_enc
0,3177,2025,2158,93,409,1,0,1,0,0.053236,0.043026,-0.001450,0,0.350266,0.04976,0.01044,0.042113,0.008360,218,36
1,487,1445,2215,270,2149,1,0,1,0,0.055525,0.032434,-0.037253,10,0.028252,0.01276,0.01688,0.008028,0.016246,262,186
2,401,545,845,180,1050,1,0,1,0,0.027747,0.066093,-0.037253,10,0.101855,0.00390,0.01688,0.004639,0.016246,24,186
3,2475,701,1157,176,1171,1,1,1,0,0.055303,0.058166,-0.001450,0,0.101855,0.01276,0.03598,0.008028,0.023283,262,79
4,5202,2245,20,95,481,1,1,1,0,0.032434,0.008157,-0.037253,10,0.350266,0.04904,0.00318,0.035133,0.003390,17,249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1113,1000,1157,117,516,1,1,12,1,0.059514,0.058166,-0.019545,3,0.225312,0.00346,0.04904,0.003075,0.035133,219,18
9996,3846,1343,1455,72,290,1,1,12,1,0.054044,0.055525,-0.001450,0,0.350266,0.00346,0.01660,0.003075,0.020192,219,158
9997,243,1445,1804,199,1089,1,1,12,1,0.055525,0.059659,-0.001450,0,0.101855,0.01660,0.01078,0.020192,0.023044,155,195
9998,801,1324,1610,166,896,1,1,12,1,0.054044,0.056478,0.001304,4,0.101855,0.00044,0.01688,0.000720,0.016246,299,186


## Standardizing features that were in original units:

In [79]:
# adding 1 to zero values in y (flights with no delays):
y_plus1 = y + 1

# standardized time and distance:
from sklearn.preprocessing import StandardScaler
#new df with standardized values
X_stscaled = X.copy()
# features to standardize
stand_feat = ['crs_elapsed_time', 'distance', 'mkt_carrier_fl_num', 'crs_dep_time', 'crs_arr_time']
# adding the standardized columns
standardized_feat = StandardScaler().fit_transform(X_stscaled[stand_feat])
X_stscaled['crs_elap_t_std'], X_stscaled['dist_std'], X_stscaled['mkt_carrier_num_std'], X_stscaled['crs_dep_time_std'], X_stscaled['crs_arr_time_std'] = [standardized_feat[:,0], standardized_feat[:,1], standardized_feat[:,2], standardized_feat[:,3], standardized_feat[:,4]]
# dropping the original values
X_stscaled = X_stscaled.drop(['crs_elapsed_time', 'distance','mkt_carrier_fl_num', 'crs_dep_time', 'crs_arr_time'], axis=1)
# dropping the useless features
X_stscaled = X_stscaled.drop(['origin_airport_freq', 'dest_airport_freq', 'season', 'weekday', 'month', 
    'arr_taxi_ind', 'airtime_ind', 'orig_airp_fl_ind', 'dest_airp_fl_ind', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

In [80]:
X_stscaled

,dep_weather_rank,arr_weather_rank,peak_season,dep_taxi_ind,carriers_lateness_ind,carrier_enc,orig_airp_enc,dest_airp_enc,crs_elap_t_std,dist_std,mkt_carrier_num_std,crs_dep_time_std,crs_arr_time_std
0,1,1,0,0.027747,-0.037253,10,23,181,0.589420,0.477810,-1.242432,-1.597293,-1.236174
1,1,1,0,0.055525,-0.037253,10,261,181,1.854207,2.358605,-1.196351,0.237929,1.416092
2,1,1,0,0.053236,-0.001450,0,217,35,-0.633207,-0.619177,0.245019,1.420628,1.305742
3,1,1,0,0.032434,-0.037253,10,16,244,-0.605101,-0.495959,1.330064,1.869238,-2.833341
4,1,1,0,0.055303,-0.037253,10,108,17,-0.324037,-0.324822,-0.710894,-1.179270,-1.042578
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3,1,1,0.059419,0.055916,8,58,211,-0.815899,-0.983699,0.963560,0.380669,0.293235
9996,1,2,1,0.054044,-0.001450,0,218,155,-0.928324,-0.822830,0.603486,0.029937,-0.055238
9997,1,1,1,0.066093,-0.019545,3,16,228,-0.562941,-0.418947,-0.715716,-0.961083,-0.868341
9998,1,1,1,0.055303,-0.001450,0,218,222,-1.054803,-0.956317,1.268980,-1.177231,-1.218750


## Small Variance:

In [101]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(0.1)
high_var_arr = vt.fit_transform(X) # features with small variance get deleted

In [102]:
high_var_arr.shape # array of the ones that remained

(10000, 12)

In [103]:
# Creating the df back
X_highv_col = X.columns[vt.get_support()] # get_support() is method of VarianceThreshold and stores boolean of each variable in the numpy array.
X_high_var = pd.DataFrame(high_var_arr, columns=X_highv_col) # transforming an array back to a data-frame preserves column labels

In [104]:
X_high_var

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,season,weekday,month,peak_season,carrier_enc,orig_airp_enc,dest_airp_enc
0,3177.0,2025.0,2158.0,93.0,409.0,1.0,0.0,1.0,0.0,0.0,218.0,36.0
1,487.0,1445.0,2215.0,270.0,2149.0,1.0,0.0,1.0,0.0,10.0,262.0,186.0
2,401.0,545.0,845.0,180.0,1050.0,1.0,0.0,1.0,0.0,10.0,24.0,186.0
3,2475.0,701.0,1157.0,176.0,1171.0,1.0,1.0,1.0,0.0,0.0,262.0,79.0
4,5202.0,2245.0,20.0,95.0,481.0,1.0,1.0,1.0,0.0,10.0,17.0,249.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1113.0,1000.0,1157.0,117.0,516.0,1.0,1.0,12.0,1.0,3.0,219.0,18.0
9996,3846.0,1343.0,1455.0,72.0,290.0,1.0,1.0,12.0,1.0,0.0,219.0,158.0
9997,243.0,1445.0,1804.0,199.0,1089.0,1.0,1.0,12.0,1.0,0.0,155.0,195.0
9998,801.0,1324.0,1610.0,166.0,896.0,1.0,1.0,12.0,1.0,4.0,299.0,186.0


In [105]:
X_high_var_list = list(X_high_var.columns)

## Correlations:

In [106]:
X.corr()[X.corr().abs() > 0.8]

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,season,weekday,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,carrier_enc,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind,orig_airp_enc,dest_airp_enc
mkt_carrier_fl_num,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crs_dep_time,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crs_arr_time,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crs_elapsed_time,NaN,NaN,NaN,1.000000,0.983192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.806754,NaN,NaN,NaN,NaN,NaN,NaN
distance,NaN,NaN,NaN,0.983192,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.804920,NaN,NaN,NaN,NaN,NaN,NaN
season,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_season,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dep_taxi_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
# dropping features with high correlations in correlation matrix X vs y:
X_non_corr = X.drop(['distance', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

# dropping features with high correlations in correlation matrix X_stscaled vs y:
X_stscaled_non_corr = X_stscaled.drop(['airtime_ind', 'dist_std', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

In [163]:
X_stscaled_non_corr.corr()[X_stscaled_non_corr.corr().abs() > 0.8]

,season,weekday,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,carrier_enc,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_enc,dest_airp_enc,crs_elap_t_std
season,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_season,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dep_taxi_ind,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
arr_taxi_ind,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carriers_lateness_ind,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
carrier_enc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
orig_airp_fl_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
dest_airp_fl_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


## Feature Selection: Forward Regression

In [107]:
from sklearn.feature_selection import f_regression, SelectKBest
skb = SelectKBest(f_regression, k=10) # instantiating the n features that best explain y
X_fwd_reg_array = skb.fit_transform(X, y) #applying algo to the data
X_fwd_reg_array.shape

(10000, 10)

In [108]:
# creating a Datarame with the values of the matrix X and the column names above
X_fwd_reg = pd.DataFrame(X, columns=X.columns[skb.get_support()]) #get support: list that holds True if the feature was selected

In [109]:
X_fwd_reg

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,dest_airp_fl_ind,dest_airp_pss_ind
0,3177,2025,2158,1,0,0.053236,0.043026,-0.001450,0.01044,0.008360
1,487,1445,2215,1,0,0.055525,0.032434,-0.037253,0.01688,0.016246
2,401,545,845,1,0,0.027747,0.066093,-0.037253,0.01688,0.016246
3,2475,701,1157,1,0,0.055303,0.058166,-0.001450,0.03598,0.023283
4,5202,2245,20,1,0,0.032434,0.008157,-0.037253,0.00318,0.003390
...,...,...,...,...,...,...,...,...,...,...
9995,1113,1000,1157,12,1,0.059514,0.058166,-0.019545,0.04904,0.035133
9996,3846,1343,1455,12,1,0.054044,0.055525,-0.001450,0.01660,0.020192
9997,243,1445,1804,12,1,0.055525,0.059659,-0.001450,0.01078,0.023044
9998,801,1324,1610,12,1,0.054044,0.056478,0.001304,0.01688,0.016246


In [110]:
X_fwd_reg_list = list(X_fwd_reg.columns)

## SKLearn Feature Selection Algo Linear Regression:

In [111]:
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(estimator=LinearRegression())
X_feat_sel_algo = selector.fit_transform(X, y)
X_feat_sel_algo.shape


(10000, 6)

In [112]:
X_feat_sel_algo_df = pd.DataFrame(X, columns=X.columns[selector.get_support()]) #get support: list that holds True if the feature was selected

In [113]:
X_feat_sel_algo_df

,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind
0,0.053236,0.043026,-0.001450,0.04976,0.01044,0.042113
1,0.055525,0.032434,-0.037253,0.01276,0.01688,0.008028
2,0.027747,0.066093,-0.037253,0.00390,0.01688,0.004639
3,0.055303,0.058166,-0.001450,0.01276,0.03598,0.008028
4,0.032434,0.008157,-0.037253,0.04904,0.00318,0.035133
...,...,...,...,...,...,...
9995,0.059514,0.058166,-0.019545,0.00346,0.04904,0.003075
9996,0.054044,0.055525,-0.001450,0.00346,0.01660,0.003075
9997,0.055525,0.059659,-0.001450,0.01660,0.01078,0.020192
9998,0.054044,0.056478,0.001304,0.00044,0.01688,0.000720


In [114]:
X_feat_sel_algo_list = list(X_feat_sel_algo_df.columns)

## SKLearn Feature Selection Algo Random Forest Regression:

In [153]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

selector = SelectFromModel(estimator=RandomForestRegressor())
X_feat_sel_algo_randf = selector.fit_transform(X, y)
X_feat_sel_algo_randf.shape


(10000, 8)

In [154]:
X_feat_sel_algo_randf_df = pd.DataFrame(X, columns=X.columns[selector.get_support()]) #get support: list that holds True if the feature was selected

In [155]:
X_feat_sel_algo_randf_df

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,dest_airp_pss_ind,orig_airp_enc,dest_airp_enc
0,3177,2025,2158,93,409,0.008360,218,36
1,487,1445,2215,270,2149,0.016246,262,186
2,401,545,845,180,1050,0.016246,24,186
3,2475,701,1157,176,1171,0.023283,262,79
4,5202,2245,20,95,481,0.003390,17,249
...,...,...,...,...,...,...,...,...
9995,1113,1000,1157,117,516,0.035133,219,18
9996,3846,1343,1455,72,290,0.020192,219,158
9997,243,1445,1804,199,1089,0.023044,155,195
9998,801,1324,1610,166,896,0.016246,299,186


In [156]:
X_feat_sel_algo_randf_list = list(X_feat_sel_algo_randf_df.columns)

# Modeling:

## Statsmodel Linear Regression:

In [22]:
# adding a constant for X
X_with_b0 = sm.add_constant(X)

# CHOOSE which features to leave out. Lists in Feature selection Section:

# redundant
X_with_b0 = X_with_b0.drop(redundant_feat, axis=1) 
# # engineered time of day related
# X_with_b0 = X_with_b0.drop(eng_time_feat, axis=1)
# # engineered location related
# X_with_b0 = X_with_b0.drop(eng_airp_feat, axis=1)
# # with low p_value
X_with_b0 = X_with_b0.drop(low_p_val_list, axis=1)
# # label encoded
# X_with_b0 = X_with_b0.drop(label_enc_feat, axis=1) 

#Remove constant to run with SKlearn:
X_no_b0 = X_with_b0.drop(['const'], axis=1)

#modeling vanilla ls:
model = sm.OLS(y, X_with_b0).fit()
print(model.summary())
# pickle.dump(model, open('Data/Pickles/vanilla_ls_allfeat_noscalling.sav', 'wb'))

                            OLS Regression Results                            
Dep. Variable:              arr_delay   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     17.21
Date:                Fri, 24 Sep 2021   Prob (F-statistic):           3.84e-37
Time:                        15:45:45   Log-Likelihood:                -52461.
No. Observations:               10000   AIC:                         1.049e+05
Df Residuals:                    9987   BIC:                         1.050e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -23.94

In [ ]:
X_no_b0

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,dep_weather_rank,arr_weather_rank,peak_season,dep_taxi_ind,carriers_lateness_ind,carrier_enc,orig_airp_enc,dest_airp_enc
0,401,545,845,180,1,1,0,0.027747,-0.037253,10,23,181
1,487,1445,2215,270,1,1,0,0.055525,-0.037253,10,261,181
2,3177,2025,2158,93,1,1,0,0.053236,-0.001450,0,217,35
3,5202,2245,20,95,1,1,0,0.032434,-0.037253,10,16,244
4,1393,750,945,115,1,1,0,0.055303,-0.037253,10,108,17
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4518,1515,1635,80,3,1,1,0.059419,0.055916,8,58,211
9996,3846,1343,1455,72,1,2,1,0.054044,-0.001450,0,218,155
9997,1384,857,1035,98,1,1,1,0.066093,-0.019545,3,16,228
9998,5088,751,854,63,1,1,1,0.055303,-0.001450,0,218,222


In [ ]:
# Model Evaluation
y_pred = model.predict(X_with_b0)
n = y.shape[0]
p = X_with_b0.shape[1]
print('R2 (adj) =', 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1))
print('RMSE =', mean_squared_error(y, y_pred))
print('MAE =', mean_absolute_error(y, y_pred))


R2 (adj) = 0.015066670006606664
RMSE = 2115.0549132510528
MAE = 20.268971239142378


## Polynomial trial:

### With SKLearn LinReg:

In [21]:
reg = LinearRegression().fit(X_poly, y)
y_pred = reg.predict(X_poly)
n = y.shape[0]
p = X.shape[1]
print('R2 (adj) =', 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1))
print('RMSE =', mean_squared_error(y, y_pred))
print('MAE =', mean_absolute_error(y, y_pred))

R2 (adj) = 0.1916973453882308
RMSE = 1736.231889204771
MAE = 22.29832946417685


In [59]:
# pickle.dump(reg, open('Data/Pickles/Poly_Grade4_Final_Feat_filtered.sav', 'wb'))

### With StatsModels:

In [20]:
from sklearn.preprocessing import PolynomialFeatures

X_poly = PolynomialFeatures(4).fit_transform(X_no_b0)
# X_poly_col = PolynomialFeatures(2).get_feature_names(X_no_b0.columns.to_numpy)

X_poly_b0 = sm.add_constant(X_poly)

#modeling vanilla ls
model = sm.OLS(y, X_poly_b0).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              arr_delay   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     2.435
Date:                Fri, 24 Sep 2021   Prob (F-statistic):          3.69e-102
Time:                        15:39:32   Log-Likelihood:                -51341.
No. Observations:               10000   AIC:                         1.047e+05
Df Residuals:                    8978   BIC:                         1.121e+05
Df Model:                        1021                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.085e-05   6.71e-06      4.601      0.0

## Random Forest Trial:

In [ ]:
# STANDARDIZING TARGET
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_no_b0, y)


RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
y_pred = regr.predict(X_no_b0)
n = y.shape[0]
p = X.shape[1]
print('R2 (adj) =', 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1))
print('RMSE =', mean_squared_error(y, y_pred))
print('MAE =', mean_absolute_error(y, y_pred))

[1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1), mean_squared_error(y, y_pred), mean_absolute_error(y, y_pred)]

R2 (adj) = 0.020693309537547666
RMSE = 2103.5480900519347
MAE = 20.373398206427325


[0.020693309537547666, 2103.5480900519347, 20.373398206427325]

# Evaluation - Collecting Metrics:

## Linear Regression Results:

In [88]:
# Creating empty df to store metrics and compare:
# lr_metrics_df = pd.DataFrame(columns=['R2_adj', 'RMSE', 'MAE'], dtype='float64')

In [15]:
#Function to run:
# trans_tar_reg_function(X_stscaled, y_plus1, LinearRegression(), np.log, np.exp)
trans_tar_reg_function(X_poly, y, LinearRegression(), None, None)


[0.014204795237510948, 1736.231889204771, 22.29832946417685]

In [84]:
# Adding row to the df:
lr_metrics_df.loc['X-std y-log +W'] = trans_tar_reg_function(X_stscaled, y_plus1, LinearRegression(), np.log, np.exp)

In [85]:
# looking df
lr_metrics_df

,R2_adj,RMSE,MAE
Original +W,0.019203,2106.538904,20.169061
W/o Redundant +W,0.019101,2106.967949,20.174780
W/o Location Eng +W,0.019061,2107.898356,20.194183
W/o Time Eng +W,0.018545,2110.065329,20.148238
W/o Label Enc +W,0.018817,2110.113982,20.154082
W/o Low p-v +W,0.018983,2109.545763,20.215144
Poly (2) +W,0.023444,2083.550540,20.050634
Poly (3) +W,0.027707,1998.243992,20.817340
Poly (4) +W,0.014161,1736.308236,22.305427
X-std y-lin +W,0.019234,2109.004881,20.207857


In [86]:
# saving df
# lr_metrics_df.to_csv('Data/Daniel Model Evaluation/Daniel Output OLS-LR Weather.csv')

## Random Forest Results:

In [ ]:
# Creating empty df to store metrics and compare:
# rf_metrics_df = pd.DataFrame(columns=['R2_adj', 'RMSE', 'MAE'], dtype='float64')

In [89]:
rf_metrics_df.loc['W/o Low p-v +W'] = [0.020693309537547666, 2103.5480900519347, 20.373398206427325]

In [90]:
rf_metrics_df#.rename(index={'X filtered, y' : 'W/o refundant'})

,R2_adj,RMSE,MAE
W/o Low p-v +W,0.020693,2103.54809,20.373398


In [222]:
# rf_metrics_df.to_csv('Data/Daniel Model Evaluation/Daniel Output RandFor.csv')

# Predictions:

In [37]:
# importing and basic formating
flights_test = pd.read_csv('C:/Users/danfv/OneDrive/Knowledge/Courses/Data Analysis/LHL - Data Analysis Bootcamp/lighthouse-data-notes/W6 Midterm Project/Midterm_Project/Data/Flights_test - Sample 100 - Peter Dan Feat.csv').sort_values('fl_date').reset_index(drop=True)

In [42]:
# Use X_no_b0 created in the Statsmodel Linear Regression section as the filter to select columns of the testong data:
X_test = flights_test[X_no_b0.columns]

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,dep_weather_rank,arr_weather_rank,peak_season,dep_taxi_ind,carriers_lateness_ind,carrier_enc,orig_airp_enc,dest_airp_enc
0,5065,1610,1715,125,1,1,0,0.056478,-0.037253,8,15,26
1,4709,1659,1808,129,1,1,0,0.056478,0.055916,7,56,10
2,3066,1639,1746,67,1,1,0,0.056478,-0.001450,0,14,14
3,3557,1021,1130,69,1,1,0,0.059514,-0.001450,0,53,11
4,64,1105,1245,100,1,2,0,0.058166,-0.025750,1,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...
95,5747,502,610,68,1,2,0,0.027747,-0.001450,0,29,11
96,2248,545,725,160,1,2,0,0.027747,-0.037253,8,3,29
97,2905,1745,2144,179,2,1,0,0.061765,-0.001450,0,32,30
98,1093,1735,2036,181,1,1,0,0.061765,-0.001450,0,34,12


In [56]:
# predicting for test
X_poly_test = PolynomialFeatures(4).fit_transform(X_test)
y_pred_test = reg.predict(X_poly_test)

In [57]:
# merging predictions with additional columns required for submission:
y_test_pred = flights_test.copy()[['fl_date','mkt_carrier','mkt_carrier_fl_num','origin','dest']]
y_test_pred['predicted_delay'] = y_pred_test
y_test_pred

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01,WN,5065,ELP,LAX,-35.274064
1,2020-01-01,UA,4709,XWA,DEN,58.441884
2,2020-01-01,AA,3066,DFW,FSM,1.012929
3,2020-01-01,AA,3557,TXK,DFW,0.622034
4,2020-01-01,AS,64,ANC,JNU,-26.430593
...,...,...,...,...,...,...
95,2020-01-30,AA,5747,LBB,DFW,70.370560
96,2020-01-30,WN,2248,BDL,MDW,-5.534423
97,2020-01-31,AA,2905,MCI,MIA,23.839860
98,2020-01-31,AA,1093,MIA,EWR,8.230961


In [58]:
# exporting CSV:
y_test_pred.to_csv('Data/Daniel Model Evaluation/Flights_test_prediction.csv')